# Severity
#### 이전연구에서 사용했던 potential varibales는 다음과 같습니다.
*SAPS II \
*APACHE \
*MIMIC-IV 는 더 다양할 수 있습니다.

In [1]:
#potential tables
#mimiciv_derived, charlson, mai_admin, None, False, False, False, False
#mimiciv_derived, apsiii, mai_admin, None, False, False, False, False
#mimiciv_derived, oasis, mai_admin, None, False, False, False, False
#mimiciv_derived, sapsii, mai_admin, None, False, False, False, False
#mimiciv_derived, sofa, mai_admin, None, False, False, False, False
#mimiciv_derived, first_day_sofa, mai_admin, None, False, False, False, False
#mimiciv_derived, sepsis3, mai_admin, None, False, False, False, False
#https://www.mdcalc.com/calc/3917/charlson-comorbidity-index-cci

In [2]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import pprint
from datetime import datetime, timedelta
#import tim
from dfply import *
import matplotlib.pyplot as plt
import seaborn as sns
import impyute as impy
from impyute.imputation.cs import mice
#from sqlalchemy import create_engine
#https://chmodi.tistory.com/114

In [3]:
import psycopg2
from sshtunnel import SSHTunnelForwarder

try:
    tunnel= SSHTunnelForwarder(
         ('210.91.223.248', 35430),
         #ssh_private_key="</path/to/private/ssh/key>",
         ### in my case, I used a password instead of a private key
         ssh_username="medicalai",
         ssh_password="medicalai731!", 
         remote_bind_address=('localhost', 35432))
         
    tunnel.start()
    print("tunnel connected")

    params = {
             'database': 'mimiciv',
             'user': 'mai_admin',
             'password': 'admin1q2w!@',
             'host': 'localhost',
             'port': tunnel.local_bind_port
             }

    conn = psycopg2.connect(**params)
    conn.autocommit = True
    curs = conn.cursor()
    print("database connected")

except Exception as e:
    print("Connection Failed due to {e}")

# Remember to close the SSH tunnel
#tunnel.close()


tunnel connected
database connected


*saps ii (derived), dasis(derived), apsiii(derived), oasis(derived)

In [4]:
curs.execute("select * from mimiciv_derived.charlson;")
columns_name = [desc[0] for desc in curs.description]
charlson = pd.DataFrame(curs.fetchall(), columns=columns_name) 

In [5]:
curs.execute("select * from mimiciv_derived.apsiii;")
columns_name = [desc[0] for desc in curs.description]
apsiii = pd.DataFrame(curs.fetchall(), columns=columns_name) 

In [6]:
curs.execute("select * from mimiciv_derived.oasis;")
columns_name = [desc[0] for desc in curs.description]
oasis = pd.DataFrame(curs.fetchall(), columns=columns_name) 

In [7]:
curs.execute("select * from mimiciv_derived.sapsii;")
columns_name = [desc[0] for desc in curs.description]
sapsii = pd.DataFrame(curs.fetchall(), columns=columns_name) 

In [8]:
curs.execute("select * from mimiciv_derived.sofa;")
columns_name = [desc[0] for desc in curs.description]
sofa = pd.DataFrame(curs.fetchall(), columns=columns_name) 

In [9]:
curs.execute("select * from mimiciv_derived.sepsis3;")
columns_name = [desc[0] for desc in curs.description]
sepsis3 = pd.DataFrame(curs.fetchall(), columns=columns_name) 

In [10]:
charlson.head(1)

,subject_id,hadm_id,age_score,myocardial_infarct,congestive_heart_failure,peripheral_vascular_disease,cerebrovascular_disease,dementia,chronic_pulmonary_disease,rheumatic_disease,...,mild_liver_disease,diabetes_without_cc,diabetes_with_cc,paraplegia,renal_disease,malignant_cancer,severe_liver_disease,metastatic_solid_tumor,aids,charlson_comorbidity_index
0,18910522,20000041,2,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,3


In [11]:
apsiii.head(1)

,subject_id,hadm_id,stay_id,apsiii,apsiii_prob,hr_score,mbp_score,temp_score,resp_rate_score,pao2_aado2_score,...,wbc_score,creatinine_score,uo_score,bun_score,sodium_score,albumin_score,bilirubin_score,glucose_score,acidbase_score,gcs_score
0,12466550,23998182,30000153,44,0.08654575921862385844,7.0,6.0,0.0,0.0,NaN,...,0.0,0.0,5.0,7.0,0.0,NaN,NaN,0.0,1.0,15.0


In [12]:
oasis.head(1)

,subject_id,hadm_id,stay_id,oasis,oasis_prob,age,age_score,preiculos,preiculos_score,gcs,...,resprate,resp_rate_score,temp,temp_score,urineoutput,urineoutput_score,mechvent,mechvent_score,electivesurgery,electivesurgery_score
0,15726063,23069026,36415879,27,0.06110557118878299662,53.3839872681065644,6,73.0000000000000000,3,15.0,...,31.0,6.0,38.56,2.0,4580.0,0.0,0,0,0,6


In [13]:
sapsii.head(1)

,subject_id,hadm_id,stay_id,starttime,endtime,sapsii,sapsii_prob,age_score,hr_score,sysbp_score,...,uo_score,bun_score,wbc_score,potassium_score,sodium_score,bicarbonate_score,bilirubin_score,gcs_score,comorbidity_score,admissiontype_score
0,14046553,20000094,35605481,2150-03-02 15:19:31,2150-03-03 15:19:31,98,0.982729,18,11.0,13.0,...,11.0,10.0,0.0,0.0,0.0,3.0,0.0,26.0,0,6


In [14]:
sofa.head(1)

,stay_id,hr,starttime,endtime,pao2fio2ratio_novent,pao2fio2ratio_vent,rate_epinephrine,rate_norepinephrine,rate_dopamine,rate_dobutamine,...,cardiovascular,cns,renal,respiration_24hours,coagulation_24hours,liver_24hours,cardiovascular_24hours,cns_24hours,renal_24hours,sofa_24hours
0,30000153,0,2174-09-29 12:00:00,2174-09-29 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,NaN,0,0,0,0,0,0,0


In [15]:
sepsis3.head(1)

,subject_id,stay_id,antibiotic_time,culture_time,suspected_infection_time,sofa_time,sofa_score,respiration,coagulation,liver,cardiovascular,cns,renal,sepsis3
0,18421337,30000484,2136-01-14 21:00:00,2136-01-14 18:10:00,2136-01-14 18:10:00,2136-01-14 19:00:00,3,0,0,0,0,3,0,True


In [16]:
# charlson.to_csv('../outputs/charlson.csv', index=False, encoding='utf-8-sig')
# apsiii.to_csv('../outputs/apsiii.csv', index=False, encoding='utf-8-sig')
# oasis.to_csv('../outputs/oasis.csv', index=False, encoding='utf-8-sig')
# sapsii.to_csv('../outputs/sapsii.csv', index=False, encoding='utf-8-sig')
# sofa.to_csv('../outputs/sofa.csv', index=False, encoding='utf-8-sig')
# sepsis3.to_csv('../outputs/sepsis3.csv', index=False, encoding='utf-8-sig')

In [23]:
charlson = pd.read_csv('../outputs/charlson.csv')
apsiii = pd.read_csv('../outputs/apsiii.csv')
oasis = pd.read_csv('../outputs/oasis.csv')
sapsii = pd.read_csv('../outputs/sapsii.csv', parse_dates=['starttime','endtime'])
sofa = pd.read_csv('../outputs/sofa.csv', parse_dates=['starttime','endtime'])
sepsis3 = pd.read_csv('../outputs/sepsis3.csv', parse_dates=['antibiotic_time','culture_time','suspected_infection_time','sofa_time'])

In [20]:
#charlson, apsiii, oasis 는 입원 시 측정하는 점수인듯
#aps iii 는 APACHE-III 이전에 생리학검사 점수임. => 계산할 필요가 있을지?https://github.com/MIT-LCP/mimic-code/issues/653
#oasis는 그대로 사용
#sapsii 도 그대로 사용
#sofa는 spo2/fio2 ratio 인듯? pao2fio2ratio_novent	pao2fio2ratio_vent meta정보가 많은데 왜 많은지 봐야할듯.
#sepsis3 는 deathcosality 와 연관해서 볼 정보같고, severity 점수로 학습에 포함하지는 않을 것.


### 각 정보 환자정보와 join

In [27]:
subjectlist1 = pd.read_csv('./data/subjectlist1.csv', parse_dates=['admittime','intubationtime','dischtime', 'deathtime','extubationtime','reintubation_eventtime'])
subjectlist1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9003 entries, 0 to 9002
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Unnamed: 0              9003 non-null   int64         
 1   subject_id              9003 non-null   int64         
 2   hadm_id                 9003 non-null   int64         
 3   int_stayid              7860 non-null   float64       
 4   admittime               9003 non-null   datetime64[ns]
 5   intubationtime          7860 non-null   datetime64[ns]
 6   int_itemid              7860 non-null   float64       
 7   int_weight              7860 non-null   float64       
 8   ext_stayid              6825 non-null   float64       
 9   extubationtime          8759 non-null   datetime64[ns]
 10  ext_itemid              6825 non-null   float64       
 11  ext_weight              6825 non-null   float64       
 12  extubationcause         6825 non-null   object  

In [29]:
sub1list = subjectlist1 >> select('subject_id','hadm_id','int_stayid')

In [39]:
sub1_charlson = sub1list >> left_join(charlson, by=['subject_id','hadm_id']) >> select('subject_id','hadm_id','int_stayid','charlson_comorbidity_index')

In [40]:
sub1_charlson.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9003 entries, 0 to 9002
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   subject_id                  9003 non-null   int64  
 1   hadm_id                     9003 non-null   int64  
 2   int_stayid                  7860 non-null   float64
 3   charlson_comorbidity_index  9003 non-null   int64  
dtypes: float64(1), int64(3)
memory usage: 281.5 KB


In [41]:
sub1_apsiii = sub1list >> left_join(apsiii, by=['subject_id','hadm_id']) >> select('subject_id','hadm_id','int_stayid','apsiii')

In [42]:
sub1_apsiii.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12672 entries, 0 to 12671
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   subject_id  12672 non-null  int64  
 1   hadm_id     12672 non-null  int64  
 2   int_stayid  10731 non-null  float64
 3   apsiii      12672 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 396.1 KB


In [45]:
sub1_oasis = sub1list >> left_join(oasis, by = ['subject_id','hadm_id']) >> select('subject_id','hadm_id','int_stayid','stay_id','oasis')

In [46]:
sub1_oasis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12672 entries, 0 to 12671
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   subject_id  12672 non-null  int64  
 1   hadm_id     12672 non-null  int64  
 2   int_stayid  10731 non-null  float64
 3   stay_id     12672 non-null  int64  
 4   oasis       12672 non-null  int64  
dtypes: float64(1), int64(4)
memory usage: 495.1 KB


In [50]:
sub1_sapsii = sub1list >> left_join(sapsii, by = ['subject_id','hadm_id']) >> select('subject_id','hadm_id','stay_id','starttime','endtime','sapsii')


In [51]:
sub1_sapsii.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12672 entries, 0 to 12671
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   subject_id  12672 non-null  int64         
 1   hadm_id     12672 non-null  int64         
 2   stay_id     12672 non-null  int64         
 3   starttime   12672 non-null  datetime64[ns]
 4   endtime     12672 non-null  datetime64[ns]
 5   sapsii      12672 non-null  int64         
dtypes: datetime64[ns](2), int64(4)
memory usage: 594.1 KB


In [64]:
#sub1list의 int_stayid 를 int64 로 변환
#copilot 코드 제안 해줘야지???
sub1list( )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9003 entries, 0 to 9002
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   subject_id  9003 non-null   int64  
 1   hadm_id     9003 non-null   int64  
 2   int_stayid  7860 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 211.1 KB
None


In [60]:
pd.merge(sub1list, sofa, how='left', left_on='int_stayid', right_on='stay_id')

,subject_id,hadm_id,int_stayid,stay_id,hr,starttime,endtime,pao2fio2ratio_novent,pao2fio2ratio_vent,rate_epinephrine,...,cardiovascular,cns,renal,respiration_24hours,coagulation_24hours,liver_24hours,cardiovascular_24hours,cns_24hours,renal_24hours,sofa_24hours
0,10001884,26184834,37510196.0,37510196.0,0.0,2131-01-11 04:00:00,2131-01-11 05:00:00,NaN,NaN,NaN,...,3.0,NaN,NaN,0.0,0.0,0.0,3.0,0.0,0.0,3.0
1,10001884,26184834,37510196.0,37510196.0,1.0,2131-01-11 05:00:00,2131-01-11 06:00:00,NaN,NaN,NaN,...,3.0,NaN,NaN,0.0,0.0,0.0,3.0,0.0,0.0,3.0
2,10001884,26184834,37510196.0,37510196.0,2.0,2131-01-11 06:00:00,2131-01-11 07:00:00,NaN,NaN,NaN,...,3.0,NaN,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0
3,10001884,26184834,37510196.0,37510196.0,3.0,2131-01-11 07:00:00,2131-01-11 08:00:00,NaN,NaN,NaN,...,3.0,0.0,NaN,0.0,0.0,0.0,3.0,0.0,0.0,3.0
4,10001884,26184834,37510196.0,37510196.0,4.0,2131-01-11 08:00:00,2131-01-11 09:00:00,NaN,NaN,NaN,...,3.0,NaN,NaN,0.0,0.0,0.0,3.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1801560,19999840,21033226,38978960.0,38978960.0,120.0,2164-09-17 09:00:00,2164-09-17 10:00:00,NaN,NaN,NaN,...,4.0,NaN,0.0,3.0,0.0,0.0,4.0,0.0,0.0,7.0
1801561,19999840,21033226,38978960.0,38978960.0,121.0,2164-09-17 10:00:00,2164-09-17 11:00:00,NaN,NaN,NaN,...,4.0,NaN,0.0,3.0,0.0,0.0,4.0,0.0,0.0,7.0
1801562,19999840,21033226,38978960.0,38978960.0,122.0,2164-09-17 11:00:00,2164-09-17 12:00:00,NaN,NaN,NaN,...,4.0,NaN,NaN,3.0,0.0,0.0,4.0,0.0,0.0,7.0
1801563,19999840,21033226,38978960.0,38978960.0,123.0,2164-09-17 12:00:00,2164-09-17 13:00:00,NaN,NaN,NaN,...,4.0,0.0,0.0,3.0,0.0,0.0,4.0,0.0,0.0,7.0


In [71]:
gcs_t1 = pd.merge(subset2, gcs, how='left', on=['subject_id', 'stay_id'])
gcs_t1

,subject_id,hadm_id,stay_id,intubationtime,extubationtime,extubationcause,reintubationtime,reintubation_duration,dischtime,deathtime,outcome_step1,outcome_step2,outcome_step3,charttime,gcs,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable
0,10001884,26184834,37510196,2131-01-11 04:30:00,2131-01-12 17:40:00,Planned Extubation,NaT,NaN,2131-01-20 05:15:00,2131-01-20 05:15:00,death,unknown,unknown,2131-01-11 08:00:00,15.0,1.0,0.0,1.0,1
1,10001884,26184834,37510196,2131-01-11 04:30:00,2131-01-12 17:40:00,Planned Extubation,NaT,NaN,2131-01-20 05:15:00,2131-01-20 05:15:00,death,unknown,unknown,2131-01-11 10:00:00,15.0,1.0,0.0,2.0,1
2,10001884,26184834,37510196,2131-01-11 04:30:00,2131-01-12 17:40:00,Planned Extubation,NaT,NaN,2131-01-20 05:15:00,2131-01-20 05:15:00,death,unknown,unknown,2131-01-11 12:00:00,15.0,5.0,0.0,3.0,1
3,10001884,26184834,37510196,2131-01-11 04:30:00,2131-01-12 17:40:00,Planned Extubation,NaT,NaN,2131-01-20 05:15:00,2131-01-20 05:15:00,death,unknown,unknown,2131-01-11 14:00:00,15.0,5.0,0.0,3.0,1
4,10001884,26184834,37510196,2131-01-11 04:30:00,2131-01-12 17:40:00,Planned Extubation,NaT,NaN,2131-01-20 05:15:00,2131-01-20 05:15:00,death,unknown,unknown,2131-01-11 16:00:00,15.0,5.0,0.0,3.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372030,19999068,21606769,30143796,2161-08-25 15:34:00,2161-08-28 13:35:00,Planned Extubation,NaT,NaN,2161-09-02 19:00:00,NaT,discharged,unknown,unknown,2161-08-30 04:00:00,14.0,6.0,4.0,4.0,0
372031,19999068,21606769,30143796,2161-08-25 15:34:00,2161-08-28 13:35:00,Planned Extubation,NaT,NaN,2161-09-02 19:00:00,NaT,discharged,unknown,unknown,2161-08-30 08:05:00,14.0,6.0,4.0,4.0,0
372032,19999068,21606769,30143796,2161-08-25 15:34:00,2161-08-28 13:35:00,Planned Extubation,NaT,NaN,2161-09-02 19:00:00,NaT,discharged,unknown,unknown,2161-08-30 09:00:00,14.0,6.0,4.0,4.0,0
372033,19999068,21606769,30143796,2161-08-25 15:34:00,2161-08-28 13:35:00,Planned Extubation,NaT,NaN,2161-09-02 19:00:00,NaT,discharged,unknown,unknown,2161-08-30 10:00:00,14.0,NaN,4.0,NaN,0


In [76]:
gcs_t1.rename(columns={'charttime':'gcstime'}, inplace=True)

In [78]:
gcs_t2 = gcs_t1[(gcs_t1['gcstime'] >= gcs_t1['intubationtime']) & (gcs_t1['gcstime'] <= gcs_t1['extubationtime'])]


In [82]:
gcs_t2['gcs'].value_counts()

gcs
15.0    150310
14.0      3754
13.0      1516
10.0      1391
11.0      1313
8.0        930
9.0        837
3.0        819
7.0        790
6.0        594
12.0       564
4.0        265
5.0        174
Name: count, dtype: int64

In [84]:
#시간차를 보겠어요
gcs_t2['timeprogress'] = (gcs_t2['gcstime'] - gcs_t2['intubationtime']).dt.total_seconds() / 3600


In [87]:
gcs_t2 >> select('subject_id','hadm_id','stay_id','intubationtime','extubationtime','gcstime','timeprogress','gcs','gcs_motor','gcs_verbal','gcs_eyes','gcs_unable')

,subject_id,hadm_id,stay_id,intubationtime,extubationtime,gcstime,timeprogress,gcs,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable
0,10001884,26184834,37510196,2131-01-11 04:30:00,2131-01-12 17:40:00,2131-01-11 08:00:00,3.500000,15.0,1.0,0.0,1.0,1
1,10001884,26184834,37510196,2131-01-11 04:30:00,2131-01-12 17:40:00,2131-01-11 10:00:00,5.500000,15.0,1.0,0.0,2.0,1
2,10001884,26184834,37510196,2131-01-11 04:30:00,2131-01-12 17:40:00,2131-01-11 12:00:00,7.500000,15.0,5.0,0.0,3.0,1
3,10001884,26184834,37510196,2131-01-11 04:30:00,2131-01-12 17:40:00,2131-01-11 14:00:00,9.500000,15.0,5.0,0.0,3.0,1
4,10001884,26184834,37510196,2131-01-11 04:30:00,2131-01-12 17:40:00,2131-01-11 16:00:00,11.500000,15.0,5.0,0.0,3.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
372015,19999068,21606769,30143796,2161-08-25 15:34:00,2161-08-28 13:35:00,2161-08-27 19:32:00,51.966667,15.0,1.0,0.0,4.0,1
372016,19999068,21606769,30143796,2161-08-25 15:34:00,2161-08-28 13:35:00,2161-08-28 00:00:00,56.433333,15.0,6.0,0.0,3.0,1
372017,19999068,21606769,30143796,2161-08-25 15:34:00,2161-08-28 13:35:00,2161-08-28 04:00:00,60.433333,15.0,6.0,0.0,3.0,1
372018,19999068,21606769,30143796,2161-08-25 15:34:00,2161-08-28 13:35:00,2161-08-28 08:00:00,64.433333,15.0,6.0,0.0,3.0,1


In [89]:
gcs_t2['stay_id'].value_counts() #stay_id 에 따라 아마 intubation-reintubation event 가 있고, gcs 이벤트가 있을건데
#이렇게 많다는 뜻임. 그래서, 

stay_id
33167720    1340
33835780    1200
39867250    1079
35692039     845
34410095     819
            ... 
33542280       3
30307541       3
36173365       2
35359687       2
36601871       2
Name: count, Length: 3482, dtype: int64

In [95]:
gcs_t2['timeprogress'].median() #69시간이 중앙값...
#107시간이 맥스값... 이걸 2시간 간격? 3시간 간격? 으로 구성한다. 그럼 1,2,3일차는 그래도 꽤 있을텐데 그 이후는 퇴원하거나 이벤트 종료인 경우가 있을 것이다.

69.01666666666667

In [97]:
gcs_t2['timeprogress'].unique

<bound method Series.unique of 0          3.500000
1          5.500000
2          7.500000
3          9.500000
4         11.500000
            ...    
372015    51.966667
372016    56.433333
372017    60.433333
372018    64.433333
372019    69.066667
Name: timeprogress, Length: 163257, dtype: float64>

In [91]:
gcs_t2['timeprogress'].describe()

count    163257.000000
mean        107.132281
std         121.416415
min           0.000000
25%          28.650000
50%          69.016667
75%         143.916667
max        1480.000000
Name: timeprogress, dtype: float64

In [93]:
gcs_t2['gcs'].describe()


count    163257.000000
mean         14.642490
std           1.548745
min           3.000000
25%          15.000000
50%          15.000000
75%          15.000000
max          15.000000
Name: gcs, dtype: float64

In [92]:
gcs_t2['stay_id'].describe()

count    1.632570e+05
mean     3.497953e+07
std      2.883448e+06
min      3.000756e+07
25%      3.254347e+07
50%      3.487023e+07
75%      3.745642e+07
max      3.999217e+07
Name: stay_id, dtype: float64

In [ ]:
#가변 길이 시퀀스 LSTM, GRU, Transformer 를 사용해서 해볼까?  -> 1차원 시퀀스로 만들어야함 아무튼 이건 나중 문제고 gcs 는 붙이긴 했음
#여기서 leastest gcs는 골라내기만 하면 됨

# Height 넣기

In [98]:
first_day_height #키는 안변해서 그대로 쓰면 됩니다.

,subject_id,stay_id,height
0,12578987,32563527,173.00
1,14859979,36134276,163.00
2,11470097,36250432,157.00
3,15312343,33258084,157.00
4,13011410,37282334,163.00
...,...,...,...
73176,11539026,30183170,None
73177,18898128,38736701,None
73178,11390883,39793803,None
73179,15288589,38988285,None


In [99]:
height_t1 = pd.merge(subset2, first_day_height, how = 'left', on = ['subject_id', 'stay_id'])

In [100]:
height_t1

,subject_id,hadm_id,stay_id,intubationtime,extubationtime,extubationcause,reintubationtime,reintubation_duration,dischtime,deathtime,outcome_step1,outcome_step2,outcome_step3,height
0,10001884,26184834,37510196,2131-01-11 04:30:00,2131-01-12 17:40:00,Planned Extubation,NaT,NaN,2131-01-20 05:15:00,2131-01-20 05:15:00,death,unknown,unknown,157.00
1,10003400,23559586,38383343,2137-08-17 21:21:00,2137-08-21 15:40:00,Planned Extubation,NaT,NaN,2137-09-02 17:05:00,2137-09-02 17:05:00,death,unknown,unknown,None
2,10004401,29988601,32773003,2144-01-27 19:00:00,2144-01-30 12:30:00,Planned Extubation,2144-01-30 13:00:00,0.500000,2144-02-06 11:45:00,NaT,reintubation,reintubation,reintubation (extubation failure),168.00
3,10005817,28661809,31316840,2135-01-06 08:33:00,2135-01-19 14:00:00,Planned Extubation,NaT,NaN,2135-01-19 18:36:00,2135-01-19 18:36:00,death,death,within 48hour death,175.00
4,10015931,22130791,37093652,2177-03-25 09:57:00,2177-03-29 13:11:00,Planned Extubation,NaT,NaN,2177-03-29 14:15:00,2177-03-29 14:15:00,death,death,within 48hour death,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4727,19989918,20256395,35582926,2175-09-23 14:15:00,2175-09-26 16:00:00,Planned Extubation,NaT,NaN,2175-09-28 17:45:00,NaT,discharged,unknown,unknown,178.00
4728,19997367,20617667,35616526,2126-04-26 20:14:00,2126-05-02 16:30:00,Planned Extubation,2126-05-05 18:20:00,73.833333,2126-05-19 14:15:00,NaT,reintubation,reintubation,reintubation (extubation non-failure),147.00
4729,19997367,20617667,35616526,2126-04-26 20:14:00,2126-05-08 17:29:00,Planned Extubation,NaT,NaN,2126-05-19 14:15:00,NaT,discharged,unknown,unknown,147.00
4730,19997367,20617667,35616526,2126-05-05 18:20:00,2126-05-08 17:29:00,Planned Extubation,NaT,NaN,2126-05-19 14:15:00,NaT,discharged,unknown,unknown,147.00


In [103]:

height

,subject_id,stay_id,charttime,height
0,10000032,39553978,2180-07-23 12:36:00,152.00
1,10001725,31205490,2110-04-11 15:52:00,157.00
2,10001884,37510196,2131-01-11 04:20:00,157.00
3,10002013,39060235,2160-05-18 10:00:00,157.00
4,10002428,33987268,2156-04-12 16:24:00,150.00
...,...,...,...,...
33469,19999287,37692584,2197-07-26 03:31:00,163.00
33470,19999287,35165301,2197-08-03 20:58:00,165.00
33471,19999297,37364566,2162-08-16 05:48:00,165.00
33472,19999442,32336619,2148-11-19 14:23:00,193.00


2023-07-01 16:20:06,021| ERROR   | Socket exception: Operation timed out (60)


In [ ]:
tunnel.close() #터널 더이상 사용하지 않을때 호출해줘야 합니다.